In [ ]:
library(tidyverse)
dat <- read.csv("/data/whi/data/main_study/csv/adh_ht_pub.csv")
dem <- read.csv("/data/whi/data/main_study/csv/dem_ctos_bio.csv")
out <- read.csv("/data/whi/data/main_study/csv/outc_adj_bio.csv")

In [ ]:
dat %>% group_by(ID) %>% filter(!is.na(ADHRATE)) %>% arrange(-ENDDY) %>% slice(1) %>% ungroup() %>% select(ID, ENDDY) -> dat_enddy

In [ ]:
dem %>% group_by(ID) %>% 
    slice(1) %>% 
    filter(HRTARM %in% c('E+P intervention', 'E+P control')) %>% 
    ungroup() %>% 
    select(ID, HRTARM) %>%
    mutate(HRTARM = ifelse(HRTARM == 'E+P intervention', 1, 0)) -> CT_ID

In [ ]:
out %>% inner_join(CT_ID, by = "ID") %>%
    inner_join(dat_enddy, by = "ID") %>%
    #mutate(ENDJAMADY = as.numeric(ENDWHIDY - (as.Date('2005-04-08')-as.Date('2002-04-30')))) %>%
    mutate(ENDJAMADY = ENDDY+1) %>%
    select(ID, HRTARM, ENDJAMADY, CABG, CABGDY, PTCA, PTCADY, PE, PEDY, DVT, DVTDY, 
           ENDMTRL, ENDMTRLDY, COLORECTAL, COLORECTALDY, BKHIP, BKHIPDY, DEATH, DEATHDY,
           CHD, CHDDY, STROKE, STROKEDY, BREAST, BREASTDY) %>%
    mutate(CABG_EVENT = ifelse(CABG == 1, ifelse(CABGDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(PTCA_EVENT = ifelse(PTCA == 1, ifelse(PTCADY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(PE_EVENT = ifelse(PE == 1, ifelse(PEDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(DVT_EVENT = ifelse(DVT == 1, ifelse(DVTDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(COLORECTAL_EVENT = ifelse(COLORECTAL == 1, ifelse(COLORECTALDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(HIP_EVENT = ifelse(BKHIP == 1, ifelse(BKHIPDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(DEATH_EVENT = ifelse(DEATH == 1, ifelse(DEATHDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(ENDMTRL_EVENT = ifelse(ENDMTRL == 1, ifelse(ENDMTRLDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(CHD_EVENT = ifelse(CHD == 1, ifelse(CHDDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(STROKE_EVENT = ifelse(STROKE == 1, ifelse(STROKEDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(BREAST_EVENT = ifelse(BREAST == 1, ifelse(BREASTDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(CABGDY = ifelse(CABG_EVENT == 1, CABGDY, ENDJAMADY)) %>%
    mutate(PTCADY = ifelse(PTCA_EVENT == 1, PTCADY, ENDJAMADY)) %>%
    mutate(PEDY = ifelse(PE_EVENT == 1, PEDY, ENDJAMADY)) %>%
    mutate(DVTDY = ifelse(DVT_EVENT == 1, DVTDY, ENDJAMADY)) %>%
    mutate(ENDMTRLDY = ifelse(ENDMTRL_EVENT == 1, ENDMTRLDY, ENDJAMADY)) %>%
    mutate(COLORECTALDY = ifelse(COLORECTAL_EVENT == 1, COLORECTALDY, ENDJAMADY)) %>%
    mutate(HIPDY = ifelse(HIP_EVENT == 1, BKHIPDY, ENDJAMADY)) %>%
    mutate(DEATHDY = ifelse(DEATH_EVENT == 1, DEATHDY, ENDJAMADY)) %>%
    mutate(CHDDY = ifelse(CHD_EVENT == 1, CHDDY, ENDJAMADY)) %>%
    mutate(STROKEDY = ifelse(STROKE_EVENT == 1, STROKEDY, ENDJAMADY)) %>%
    mutate(CP_EVENT = (CABG_EVENT+PTCA_EVENT > 0)*1) %>% 
    mutate(BREASTDY = ifelse(BREAST_EVENT == 1, BREASTDY, ENDJAMADY)) %>%
    mutate(CPDY = ifelse(CABG_EVENT == PTCA_EVENT, pmin(CABGDY, PTCADY), CABGDY*CABG_EVENT + PTCADY*PTCA_EVENT)) %>%
    mutate(GLBL_EVENT = (DEATH_EVENT + COLORECTAL_EVENT + HIP_EVENT + ENDMTRL_EVENT + CHD_EVENT + STROKE_EVENT + PE_EVENT + BREAST_EVENT > 0)*1) %>%
    mutate(GLBLDY = ifelse(GLBL_EVENT == 1, 
                           pmin(ifelse(DEATH_EVENT, DEATHDY, NA),
                                ifelse(COLORECTAL_EVENT, COLORECTALDY, NA),
                                ifelse(HIP_EVENT, HIPDY, NA),
                                ifelse(ENDMTRL_EVENT, ENDMTRLDY, NA),
                                ifelse(CHD_EVENT, CHDDY, NA),
                                ifelse(STROKE_EVENT, STROKEDY, NA),
                                ifelse(PE_EVENT, PEDY, NA),
                                ifelse(BREAST_EVENT, BREASTDY, NA), na.rm = T), 
                           pmin(DEATHDY,
                                COLORECTALDY,
                                HIPDY,
                                ENDMTRLDY,
                                CHDDY,
                                STROKEDY,
                                PEDY,
                                BREASTDY)))-> out_dz

In [ ]:
colSums(out_dz[,c('DEATH_EVENT','COLORECTAL_EVENT','HIP_EVENT','CP_EVENT','ENDMTRL_EVENT','PE_EVENT','DVT_EVENT', 'CHD_EVENT', 'BREAST_EVENT', 'STROKE_EVENT', 'GLBL_EVENT')])
table(out_dz$PE_EVENT,out_dz$HRTARM)

In [ ]:
out_dz %>% summarize(CABG = mean((CABG_EVENT == 0)*(CABGDY < 365*2), na.rm=T),
                     PTCA = mean((PTCA_EVENT == 0)*(PTCADY < 365*2), na.rm=T),
                     CP = mean((CP_EVENT == 0)*(CPDY < 365*2), na.rm=T),
                     HIP = mean((HIP_EVENT == 0)*(HIPDY < 365*2), na.rm=T),
                     ENDMTRL = mean((ENDMTRL_EVENT == 0)*(ENDMTRLDY < 365*2), na.rm=T),
                     PE = mean((PE_EVENT == 0)*(PEDY < 365*2), na.rm=T),
                     DVT = mean((DVT_EVENT == 0)*(DVTDY < 365*2), na.rm=T),
                     BREAST = mean((BREAST_EVENT == 0)*(BREASTDY < 365*2), na.rm=T),
                     CHD = mean((CHD_EVENT == 0)*(CHDDY < 365*2), na.rm=T),
                     STROKE = mean((STROKE_EVENT == 0)*(STROKEDY < 365*2), na.rm=T),
                     GLBL = mean((GLBL_EVENT == 0)*(GLBLDY < 365*2), na.rm=T),
                     CABG_E = mean((CABG_EVENT ==1)*(CABGDY < 365*2), na.rm=T),
                     PTCA_E = mean((PTCA_EVENT ==1)*(PTCADY < 365*2), na.rm=T),
                     CP_E = mean((CP_EVENT ==1)*(CPDY < 365*2), na.rm=T),
                     HIP_E = mean((HIP_EVENT ==1)*(HIPDY < 365*2), na.rm=T),
                     ENDMTRL_E = mean((ENDMTRL_EVENT ==1)*(ENDMTRLDY < 365*2), na.rm=T),
                     PE_E = mean((PE_EVENT ==1)*(PEDY < 365*2), na.rm=T),
                     DVT_E = mean((DVT_EVENT ==1)*(DVTDY < 365*2), na.rm=T),
                     BREAST_E = mean((BREAST_EVENT ==1)*(BREASTDY < 365*2), na.rm=T),
                     CHD_E = mean((CHD_EVENT ==1)*(CHDDY < 365*2), na.rm=T),
                     STROKE_E = mean((STROKE_EVENT ==1)*(STROKEDY < 365*2), na.rm=T),
                     GLBL_E = mean((GLBL_EVENT ==1)*(GLBLDY < 365*2), na.rm=T)) -> temp

t(temp)

In [ ]:
library(survival)
mod <- coxph(Surv(PEDY, PE_EVENT) ~ HRTARM, data = out_dz)
summary(mod)

In [ ]:
unique((dem %>% filter(OSFLAG == 'Yes'))$ID) -> OS_ID_pre1

In [ ]:
f2 <- read.csv("/data/whi/data/main_study/csv/f2_ctos_bio.csv")

In [ ]:
unique((f2 %>% filter(HYST == 'No', ID %in% OS_ID_pre1))$ID) -> OS_ID_pre2

In [ ]:
f43 <- read.csv("/data/whi/data/main_study/csv/f43_ctos_bio.csv")

In [ ]:
(f43 %>% filter((TOTPSTAT == 'Never used') | (TOTPSTAT == 'Past user')) %>% filter(ID %in% OS_ID_pre2))$ID -> pre_cohort_ID

In [ ]:
f48 <- read.csv("/data/whi/data/main_study/csv/f48_av1_os_pub.csv")

In [ ]:
(f48 %>% filter(ID %in% pre_cohort_ID) 
    %>% filter(((ELSTYR == 'Yes') & (PLSTYR == 'Yes')) | (HRTCMBP == 'Yes')))$ID -> OS_treat_ID
(f48 %>% filter(ID %in% pre_cohort_ID) 
     %>% filter((ELSTYR == 'No') & (PLSTYR == 'No')))$ID -> OS_control_none_ID 
(f48 %>% filter(ID %in% pre_cohort_ID) 
     %>% filter(((ELSTYR == 'Yes') & (PLSTYR == 'No')) |  ((ELSTYR == 'No') & (PLSTYR == 'Yes'))))$ID -> OS_control_one_ID 


In [ ]:
length(pre_cohort_ID)

In [ ]:
length(OS_treat_ID)
length(OS_control_none_ID)
length(OS_control_one_ID)

In [ ]:
OS_ID <- data.frame(rbind(cbind(OS_treat_ID, 1), cbind(OS_control_none_ID, 0), cbind(OS_control_one_ID, -1)))
names(OS_ID) <- c("ID", "HRTARM")

In [ ]:
out <- read.csv("/data/whi/data/main_study/csv/outc_adj_bio.csv")

In [ ]:
out %>% inner_join(OS_ID, by = "ID") %>%
    mutate(ENDJAMADY = 5.2*365) %>%
    select(ID, HRTARM, ENDJAMADY, CABG, CABGDY, PTCA, PTCADY, PE, PEDY, DVT, DVTDY, 
           ENDMTRL, ENDMTRLDY, COLORECTAL, COLORECTALDY, BKHIP, BKHIPDY, DEATH, DEATHDY,
           CHD, CHDDY, STROKE, STROKEDY, BREAST, BREASTDY) %>%
    mutate(CABG_EVENT = ifelse(CABG == 1, ifelse(CABGDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(PTCA_EVENT = ifelse(PTCA == 1, ifelse(PTCADY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(PE_EVENT = ifelse(PE == 1, ifelse(PEDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(DVT_EVENT = ifelse(DVT == 1, ifelse(DVTDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(COLORECTAL_EVENT = ifelse(COLORECTAL == 1, ifelse(COLORECTALDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(HIP_EVENT = ifelse(BKHIP == 1, ifelse(BKHIPDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(DEATH_EVENT = ifelse(DEATH == 1, ifelse(DEATHDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(ENDMTRL_EVENT = ifelse(ENDMTRL == 1, ifelse(ENDMTRLDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(CHD_EVENT = ifelse(CHD == 1, ifelse(CHDDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(STROKE_EVENT = ifelse(STROKE == 1, ifelse(STROKEDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(BREAST_EVENT = ifelse(BREAST == 1, ifelse(BREASTDY <= ENDJAMADY, 1, 0), 0)) %>%
    mutate(CABGDY = ifelse(CABG_EVENT == 1, CABGDY, ENDJAMADY)) %>%
    mutate(PTCADY = ifelse(PTCA_EVENT == 1, PTCADY, ENDJAMADY)) %>%
    mutate(PEDY = ifelse(PE_EVENT == 1, PEDY, ENDJAMADY)) %>%
    mutate(DVTDY = ifelse(DVT_EVENT == 1, DVTDY, ENDJAMADY)) %>%
    mutate(ENDMTRLDY = ifelse(ENDMTRL_EVENT == 1, ENDMTRLDY, ENDJAMADY)) %>%
    mutate(COLORECTALDY = ifelse(COLORECTAL_EVENT == 1, COLORECTALDY, ENDJAMADY)) %>%
    mutate(HIPDY = ifelse(HIP_EVENT == 1, BKHIPDY, ENDJAMADY)) %>%
    mutate(DEATHDY = ifelse(DEATH_EVENT == 1, DEATHDY, ENDJAMADY)) %>%
    mutate(CHDDY = ifelse(CHD_EVENT == 1, CHDDY, ENDJAMADY)) %>%
    mutate(STROKEDY = ifelse(STROKE_EVENT == 1, STROKEDY, ENDJAMADY)) %>%
    mutate(CP_EVENT = (CABG_EVENT+PTCA_EVENT > 0)*1) %>% 
    mutate(BREASTDY = ifelse(BREAST_EVENT == 1, BREASTDY, ENDJAMADY)) %>%
    mutate(CPDY = ifelse(CABG_EVENT == PTCA_EVENT, pmin(CABGDY, PTCADY), CABGDY*CABG_EVENT + PTCADY*PTCA_EVENT)) %>%
    mutate(GLBL_EVENT = (DEATH_EVENT + COLORECTAL_EVENT + HIP_EVENT + ENDMTRL_EVENT + CHD_EVENT + STROKE_EVENT + PE_EVENT + BREAST_EVENT > 0)) %>%
    mutate(GLBLDY = ifelse(GLBL_EVENT == 1, 
                           pmin(ifelse(DEATH_EVENT, DEATHDY, NA),
                                ifelse(COLORECTAL_EVENT, COLORECTALDY, NA),
                                ifelse(HIP_EVENT, HIPDY, NA),
                                ifelse(ENDMTRL_EVENT, ENDMTRLDY, NA),
                                ifelse(CHD_EVENT, CHDDY, NA),
                                ifelse(STROKE_EVENT, STROKEDY, NA),
                                ifelse(PE_EVENT, PEDY, NA),
                                ifelse(BREAST_EVENT, BREASTDY, NA), na.rm = T), 
                           pmin(DEATHDY,
                                COLORECTALDY,
                                HIPDY,
                                ENDMTRLDY,
                                CHDDY,
                                STROKEDY,
                                PEDY,
                                BREASTDY)))-> out_dz_os

In [ ]:
out_dz_os

In [ ]:
colSums(out_dz_os[,c('DEATH_EVENT','CP_EVENT','COLORECTAL_EVENT','HIP_EVENT','CP_EVENT','ENDMTRL_EVENT','PE_EVENT','DVT_EVENT', 'CHD_EVENT', 'BREAST_EVENT', 'STROKE_EVENT', 'GLBL_EVENT')])
table(out_dz_os$PE_EVENT,out_dz_os$HRTARM)

In [ ]:
library(survival)
mod <- coxph(Surv(PTCADY, PTCA_EVENT) ~ (HRTARM == 1), data = out_dz)
summary(mod)

In [ ]:
out_dz$OS <- 0
out_dz_os$OS <- 1
final_out <- rbind(out_dz, out_dz_os)

#write.csv(final_out, '/data/whi/data/main_study/processed/ctos_table.csv', row.names = F)

final_out <- final_out %>% mutate(grp = ifelse(OS == 1, ifelse(HRTARM == 1, 'OS-TRT', 'OS-CTRL'), 
                                               ifelse(HRTARM == 1, 'CT-TRT','CT-CTRL')))

In [ ]:
library(tidyverse)
data <- read.csv("/data/whi/data/main_study/processed/merged.csv")

In [ ]:
final_out <- read.csv("/data/whi/data/main_study/processed/merged.csv")

In [ ]:
final_out %>% mutate(GLBL_E = ifelse((GLBL_DY > 7*365), 0, GLBL_E), GLBL_DY = pmin(7*365, GLBL_DY)) -> final_out0
sum(final_out0$GLBL_E, na.rm = T)/nrow(final_out0)
final_out0 %>% mutate(grp = case_when(
    (OS == 0) & (HRTARM == 0) ~ 'CT_CTRL',
    (OS == 0) & (HRTARM == 1) ~ 'CT_TRT',
    (OS == 1) & (HRTARM == 0) ~ 'OS_CTRL',
    (OS == 1) & (HRTARM == 1) ~ 'OS_TRT'
)) -> final_out0

In [ ]:
library(survival)
library(ggplot2)
library(ggfortify)
library(survminer)
final_out0 %>% mutate(CANDDY = GLBL_DY, CAND_EVENT = GLBL_E) -> final_out2
# mod <- survfit(Surv(CANDDY, CAND_EVENT)~grp, data = final_out2 %>% mutate(newday = ifelse((OS == 0) & (CANDDY > 7*365), 7*365, CANDDY)/365, 
                                                                               #CAND_EVENT = ifelse((OS == 0) & (CANDDY > 7*365), 0, CAND_EVENT),
                                                                                #CANDDY = newday) %>% filter(OS == 0))

mod <- survfit(Surv(CANDDY, CAND_EVENT)~grp, data = final_out2 %>% mutate(CANDDY = CANDDY/365))
mod
# autoplot(mod, conf.int = F) + 
#  labs(x = "\n Survival Time (Days) ", y = "Survival Probabilities \n", 
#  title = "Survival") + 
#  theme(plot.title = element_text(hjust = 0.5), 
#  axis.title.x = element_text(face="bold", colour="#FF7A33", size = 12),
#  axis.title.y = element_text(face="bold", colour="#FF7A33", size = 12),
#  legend.title = element_text(face="bold", size = 10))

ggsurvplot(mod, conf.int = F,
           risk.table = T, risk.table.col = "strata",
           fun = "event", ylim = c(0,0.1))

In [ ]:
CT_tab <- final_out0 %>% filter(OS == 0)
chisq.test(table(CT_tab$HRTARM, CT_tab$GLBL_E))

In [ ]:
for(i in names(final_out0)){
    final_out0$var <- final_out0[[i]]
    pval <- summary(coxph(Surv(GLBL_DY, GLBL_E)~grp*var, data = final_out0 %>% filter(OS == 0)))$coefficients[3,5]
    if(!is.na(pval)) if(pval < 0.2) cat(i, pval, '\n')
}

In [ ]:
pval

In [ ]:
names(final_out0)

In [ ]:
os <- final_out0 %>% filter(OS == 1)
ct <- final_out0 %>% filter(OS == 0)

table(os[,'ALCNOW_Yes'], os[,'PREG_Yes'])
table(ct[,'ALCNOW_Yes'], ct[,'PREG_Yes'])